In [ ]:
# default_exp tst_layers

# Layers_tests

> Basic layers for constructor.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import torch
from collections import OrderedDict

# ConvLayer

In [ ]:
# export
act_fn = nn.ReLU(inplace=True)

class ConvLayer(nn.Sequential):
    """Basic conv layers block"""
    def __init__(self, ni, nf, ks=3, stride=1, 
            act=True,  act_fn=act_fn, 
            bn_layer=True, bn_1st=True, zero_bn=False, 
            padding=None, bias=False, groups=1, **kwargs):

        self.act = act
        if padding==None: padding = ks//2  
        layers = [('conv', nn.Conv2d(ni, nf, ks, stride=stride, padding=padding, bias=bias, groups=groups))]
        act_bn = [('act_fn', act_fn)] if act else []
        if bn_layer:
            bn = nn.BatchNorm2d(nf)
            nn.init.constant_(bn.weight, 0. if zero_bn else 1.) 
            act_bn += [('bn', bn)]
        if bn_1st: act_bn.reverse()
        layers += act_bn
        super().__init__(OrderedDict(layers))

In [ ]:
conv_layer = ConvLayer(32, 64)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, act=False)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, bn_layer=False)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (act_fn): ReLU(inplace=True)
)

In [ ]:
conv_layer = ConvLayer(32, 64, bn_1st=True)
conv_layer

ConvLayer(
  (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act_fn): ReLU(inplace=True)
)

# Flatten

In [ ]:
# export
class Flatten(nn.Module):
    '''flat x to vector'''
    def __init__(self):
        super().__init__()
    def forward(self, x): return x.view(x.size(0), -1)

# Noop

In [ ]:
# export
class Noop(nn.Module): # alternative name Merge
    '''Dummy module for vizualize skip conn'''
    def __init__(self):
        super().__init__()
        
    def forward(self, x):
        return x

# BasicBlock

In [ ]:
# export
def DownsampleBlock(conv_layer, ni, nf, ks, stride, act=False,  **kwargs):
    '''Base downsample for res-like blocks'''
    return conv_layer(ni, nf, ks, stride, act, **kwargs)

class BasicConvBlock(nn.Sequential):
    '''Basic block of conv layers for resblock'''
    def __init__(self, conv_layer,ni,nh,nf,stride,zero_bn,**kwargs):
        super().__init__(OrderedDict([
            ('conv_0', conv_layer(ni, nh, 3, stride=stride, **kwargs)),
            ('conv_1', conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, **kwargs))
                                ]))


class BasicBlock(nn.Module):
    """Basic block (simplified) as in pytorch resnet"""
    def __init__(self, ni, nf,  expansion=1, stride=1,
                 bn_1st=True, zero_bn=False, 
#                  groups=1, base_width=64, dilation=1, norm_layer=None
                conv_layer=ConvLayer, downsample_block=DownsampleBlock,
                 **kwargs):
        super().__init__()
        self.downsample = not ni==nf or stride==2
        self.conv = BasicConvBlock(conv_layer,ni,nf,nf,stride,zero_bn,**kwargs)
        if self.downsample:
            self.downsample = downsample_block(conv_layer, ni, nf, ks=1, stride=stride, act=False, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn
                         
    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
b_block = BasicBlock(64,64)
b_block

BasicBlock(
  (conv): BasicConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([8, 64, 32, 32])

In [ ]:
b_block = BasicBlock(64,64, stride=2)
b_block

BasicBlock(
  (conv): BasicConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([8, 64, 16, 16])

In [ ]:
b_block = BasicBlock(64,128, stride=2)
b_block

BasicBlock(
  (conv): BasicConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([8, 128, 16, 16])

# Bottleneck

In [ ]:
# export

class BottleConvBlock(nn.Sequential):
    '''Basic block of conv layers for resblock'''
    def __init__(self, conv_layer,ni,nh,nf,stride,zero_bn,act=False,**kwargs):
        super().__init__(OrderedDict([
            ('conv_0', conv_layer(ni, nh, 1,                        **kwargs)),
            ('conv_1', conv_layer(nh, nh, 3, stride=stride,         **kwargs)),
            ('conv_2', conv_layer(nh, nf, 1, zero_bn=zero_bn, act=act, **kwargs))
                                ]))

class Bottleneck(nn.Module):
    '''Bottlneck block for resnet models'''
    def __init__(self, ni, nh, expansion=4, stride=1, 
                 zero_bn=False,  # bn_1st=False, 
                conv_layer=ConvLayer, act_fn=act_fn,
                 downsample_block=DownsampleBlock, **kwargs):
        super().__init__()
        self.downsample = not ni==nh or stride==2
        ni = ni*expansion
        nf = nh*expansion
        self.conv = BottleConvBlock(conv_layer,ni,nh,nf,stride,zero_bn,**kwargs)
        if self.downsample:
            self.downsample = downsample_block(conv_layer, ni, nf, ks=1, stride=stride, act=False, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn

    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
b_block = Bottleneck(16,64)
b_block

Bottleneck(
  (conv): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([8, 256, 32, 32])

In [ ]:
b_block = Bottleneck(64,64)
b_block

Bottleneck(
  (conv): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(256, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
bs = 8
xb = torch.randn(bs, 256, 32, 32)
y = b_block(xb)
y.shape

torch.Size([8, 256, 32, 32])

In [ ]:
b_block = Bottleneck(16,64, act_fn=nn.LeakyReLU())
b_block

Bottleneck(
  (conv): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): LeakyReLU(negative_slope=0.01)
)

# tests

In [ ]:
# # basic
# ('conv_0', conv_layer(ni, nf, stride=stride,   bn_1st=bn_1st, **kwargs)),
# ('conv_1', conv_layer(nf, nf, zero_bn=zero_bn, bn_1st=bn_1st, **kwargs))

In [ ]:
# # xresnet
# ('conv_0', conv_layer(ni, nh, 3, stride=stride,              **kwargs)),
# ('conv_1', conv_layer(nh, nf, 3, zero_bn=zero_bn, act=False, **kwargs))     

In [ ]:
#  # bottle
# ('conv_0', conv_layer(ni, nh, ks=1,                  bn_1st=bn_1st, **kwargs)),
# ('conv_1', conv_layer(nh, nh, stride=stride,         bn_1st=bn_1st, **kwargs)),
# ('conv_2', conv_layer(nh, nf, ks=1, zero_bn=zero_bn, bn_1st=bn_1st, **kwargs))
     

In [ ]:
# # xresnet
# ('conv_0', conv_layer(ni, nh, 1,                        **kwargs)),
# ('conv_1', conv_layer(nh, nh, 3, stride=stride,         **kwargs)),
# ('conv_2', conv_layer(nh, nf, 1, zero_bn=zero_bn, act=False, **kwargs))
        

# ResBlock

As in fastai v1.

In [ ]:
# export
class DownsampleLayer(nn.Sequential):
    """Downsample layer for Xresnet Resblock"""
    def __init__(self, ni, nf, stride=1, 
                 conv_layer=ConvLayer, act_fn=act_fn, 
                 pool=nn.AvgPool2d(2, ceil_mode=True), pool_1st=True,
                 **kwargs):
        layers  = [] if stride==1 else [('pool', pool)]
        layers += [] if ni==nf else [('idconv', conv_layer(ni, nf, 1, act=False, **kwargs))]
        if not pool_1st: layers.reverse()
        super().__init__(OrderedDict(layers))

        
class ResBlock(nn.Module):
    def __init__(self, ni, nh, expansion=1, stride=1, conv_layer=ConvLayer, 
                 act_fn=act_fn, zero_bn=True, **kwargs):
        super().__init__()
        nf,ni = nh*expansion,ni*expansion
        self.convs = BasicConvBlock(conv_layer,ni,nh,nf,stride,zero_bn,**kwargs)\
                            if expansion==1 \
                            else BottleConvBlock(conv_layer,ni,nh,nf,stride,zero_bn,**kwargs)

        self.identity = DownsampleLayer(ni, nf, stride, **kwargs) if ni!=nf or stride==2 else Noop()       
        self.merge = Noop() # us it to visualize in repr residual connection
        self.act_fn = act_fn

    def forward(self, x): return self.act_fn(self.merge(self.convs(x) + self.identity(x)))

In [ ]:
res_block = ResBlock(16,64,expansion=4, stride=2)

In [ ]:
res_block

ResBlock(
  (convs): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (identity): DownsampleLayer(
    (pool): AvgPool2d(kernel_size=2, stride=2, padding=0)
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_s

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = res_block(xb)
y.shape

torch.Size([8, 256, 16, 16])

In [ ]:
assert y.shape==torch.Size([bs, 256, 16, 16])

In [ ]:
res_block = ResBlock(16,64,expansion=4, stride=2, pool_1st=False)

In [ ]:
res_block

ResBlock(
  (convs): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (identity): DownsampleLayer(
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (pool): AvgPool2d(kernel_size=2, stri

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = res_block(xb)
y.shape

torch.Size([8, 256, 16, 16])

In [ ]:
assert y.shape==torch.Size([bs, 256, 16, 16])

In [ ]:
res_block = ResBlock(16,64,expansion=4, stride=2, pool=nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [ ]:
res_block

ResBlock(
  (convs): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (identity): DownsampleLayer(
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1,

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = res_block(xb)
y.shape

torch.Size([8, 256, 16, 16])

In [ ]:
assert y.shape==torch.Size([bs, 256, 16, 16])

In [ ]:
res_block = ResBlock(16,64,expansion=4)

In [ ]:
res_block

ResBlock(
  (convs): BottleConvBlock(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (conv_2): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (identity): DownsampleLayer(
    (idconv): ConvLayer(
      (conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): Noop()
  (act_fn): ReLU(in

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = res_block(xb)
y.shape

torch.Size([8, 256, 32, 32])

In [ ]:
assert y.shape==torch.Size([bs, 256, 32, 32])

In [ ]:
res_block = ResBlock(64,64,expansion=1)

In [ ]:
bs = 8
xb = torch.randn(bs, 64, 32, 32)
y = res_block(xb)
y.shape

torch.Size([8, 64, 32, 32])

In [ ]:
assert y.shape==torch.Size([bs, 64, 32, 32])

# end
model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted 80_test_layers.ipynb.
Converted index.ipynb.
